In [41]:
import re
import pandas as pd

# ========= PATHS =========
predictions_csv = "../results/20251107/results_faces_accounting_20251107.csv"
survey_xlsx     = "../survey/Survey_Cafe_7-11-2025.xlsx"
output_csv      = "../results/20251107/results_faces_accounting_20251107_final.csv"

# ========= LOAD =========
pred_df   = pd.read_csv(predictions_csv)
survey_df = pd.read_excel(survey_xlsx)  # first sheet by default
print(f"Loaded {len(pred_df)} prediction rows, {len(survey_df)} survey rows.")

# ========= NORMALIZE SURVEY COLUMN NAMES =========
def normalize_col(c: str) -> str:
    c = str(c).strip()
    # collapse multiple spaces and replace non-breaking spaces
    c = re.sub(r"\s+", " ", c.replace("\xa0", " "))
    # normalize unicode dashes to hyphen-minus '-'
    c = c.replace("–","-").replace("—","-").replace("-","-")
    return c

survey_df.columns = [normalize_col(c) for c in survey_df.columns]
print("Survey cols after normalize:", list(survey_df.columns))

# ========= FIND/RENAME INVOICE COLUMN TO 'InvNo' =========
inv_candidates = [c for c in survey_df.columns if "inv" in c.lower()]
if not inv_candidates:
    raise KeyError("Could not find an invoice column (e.g. 'Inv No' / 'InvNo').")
inv_col = inv_candidates[0]
if inv_col != "InvNo":
    survey_df.rename(columns={inv_col: "InvNo"}, inplace=True)

# ========= RENAME SEX/TIME COLUMNS FOR CONSISTENCY =========
rename_map = {}
if "Sex" in survey_df.columns:  rename_map["Sex"]  = "survey_gender"
if "Time" in survey_df.columns: rename_map["Time"] = "survey_time"
survey_df.rename(columns=rename_map, inplace=True)

# ========= CANONICAL AGE HEADERS WE EXPECT AFTER NORMALIZATION =========
# We'll pick whichever of these exist in the sheet.
canonical_age_headers = ["1-10","11-20","21-30","31-40","41-55","56-65","66-80","80+"]
age_cols = [c for c in survey_df.columns if c in canonical_age_headers]
if not age_cols:
    # Fallback: regex discover columns that look like ranges like "31-40" or "80+"
    age_cols = [c for c in survey_df.columns if re.match(r"^\s*\d+\s*-\s*\d+\s*$", c) or c.strip().endswith("+")]
if not age_cols:
    raise KeyError("Could not detect age columns in the survey sheet.")

# ========= COMPUTE survey_age_group FROM 'X' MARKS =========
def pick_age_group(row):
    for col in age_cols:
        val = str(row[col]).strip().upper()
        if val == "X":   # your sheet uses 'X'
            return col
    return None

survey_df["survey_age_group"] = survey_df.apply(pick_age_group, axis=1)

# ========= KEEP ONLY WHAT WE NEED & COERCE TYPES =========
need_cols = ["InvNo", "survey_gender", "survey_age_group"]
if "survey_time" in survey_df.columns:
    need_cols.append("survey_time")
survey = survey_df[need_cols].copy()

# Ensure InvNo is numeric and comparable
survey["InvNo"] = pd.to_numeric(survey["InvNo"], errors="coerce")

# ========= MAKE SURE PREDICTIONS ALSO HAVE InvNo =========
if "InvNo" not in pred_df.columns:
    # Try to extract from AccoDocNo like "224673/155" -> 155
    if "AccoDocNo" in pred_df.columns:
        pred_df["InvNo"] = pred_df["AccoDocNo"].astype(str).str.extract(r"/(\d+)", expand=False)
    else:
        raise KeyError("pred_df has no 'InvNo' and no 'AccoDocNo' to extract it from.")
pred_df["InvNo"] = pd.to_numeric(pred_df["InvNo"], errors="coerce")

# ========= OPTIONAL: STANDARDIZE GENDER TEXT =========
if "survey_gender" in survey.columns:
    survey["survey_gender"] = (
        survey["survey_gender"]
        .astype(str).str.strip().str.upper()
        .replace({"M":"Male","MALE":"Male","F":"Female","FEMALE":"Female"})
        .where(survey["survey_gender"].notna(), None)
    )

# ========= MERGE (MANY ITEMS -> ONE SURVEY) =========
merged = pred_df.merge(survey, on="InvNo", how="left")

# ========= RULE-BASED RESOLUTION WHEN MULTIPLE SURVEYS PER INVOICE =========
# We pick the survey respondent that best matches predicted_gender + predicted_age.
def best_survey_for_item(row):
    inv = row["InvNo"]
    sub = survey[survey["InvNo"] == inv]
    if sub.empty:
        return pd.Series({"survey_gender": None, "survey_age_group": None, "survey_time": None})
    if len(sub) == 1:
        out = sub.iloc[0]
        return pd.Series({"survey_gender": out.get("survey_gender"),
                          "survey_age_group": out.get("survey_age_group"),
                          "survey_time": out.get("survey_time") if "survey_time" in sub.columns else None})
    sub = sub.copy()
    sub["score"] = 0
    if "predicted_gender" in row:
        sub.loc[sub["survey_gender"] == row["predicted_gender"], "score"] += 1
    if "predicted_age" in row:
        sub.loc[sub["survey_age_group"] == row["predicted_age"], "score"] += 1
    best = sub.sort_values(["score"], ascending=False).iloc[0]
    return pd.Series({"survey_gender": best.get("survey_gender"),
                      "survey_age_group": best.get("survey_age_group"),
                      "survey_time": best.get("survey_time") if "survey_time" in sub.columns else None})

# Overwrite with the chosen respondent (row-wise)
over = merged.apply(best_survey_for_item, axis=1)
for c in ["survey_gender","survey_age_group","survey_time"]:
    if c in over.columns:
        merged[c] = over[c]

merged["survey_completed"] = merged["survey_gender"].notna()

# ========= SAVE & REPORT =========
merged.to_csv(output_csv, index=False)
print(f"\n✅ Saved merged file: {output_csv}")
print(f"Rows with survey: {merged['survey_completed'].sum()} / {len(merged)}")

print("\n— Sanity checks —")
print("Unique invoices in survey:", survey["InvNo"].nunique())
print("InvNo nulls in pred_df:", pred_df["InvNo"].isna().sum())
print("InvNo nulls in survey:", survey["InvNo"].isna().sum())
print("\nSurvey gender counts:\n", merged.loc[merged["survey_completed"], "survey_gender"].value_counts(dropna=False))
print("\nSurvey age group counts:\n", merged.loc[merged["survey_completed"], "survey_age_group"].value_counts(dropna=False))


Loaded 735 prediction rows, 61 survey rows.
Survey cols after normalize: ['Time', 'InvNo', 'Sex', '1-10', '11-20', '21-30', '31-40', '41-55', '56-65', '66-80', '80+']

✅ Saved merged file: ../results/20251107/results_faces_accounting_20251107_final.csv
Rows with survey: 384 / 894

— Sanity checks —
Unique invoices in survey: 45
InvNo nulls in pred_df: 24
InvNo nulls in survey: 0

Survey gender counts:
 survey_gender
Female    300
Male       84
Name: count, dtype: int64

Survey age group counts:
 survey_age_group
56-65    109
31-40     96
66-80     85
41-55     55
21-30     18
80+       18
11-20      3
Name: count, dtype: int64


In [ ]:
# import pandas as pd
# import re
# import chardet

# # === PATHS ===
# predictions_csv = "../results/20251107/results_faces_accounting_20251107_vit.csv"
# survey_xlsx = "../survey/Survey_Cafe_5-11-2025.xlsx"
# output_csv = "../results/20251107/results_faces_accounting_20251107_vit_final.csv"

# # === SAFE LOAD: AUTO-DETECT ENCODING ===
# with open(predictions_csv, "rb") as f:
#     enc = chardet.detect(f.read())["encoding"]
# print(f"🔍 Detected encoding for predictions CSV: {enc}")

# pred_df = pd.read_csv(predictions_csv, encoding=enc, on_bad_lines="skip", low_memory=False)
# survey_df = pd.read_excel(survey_xlsx)

# print(f"✅ Loaded {len(pred_df)} prediction records (items) and {len(survey_df)} survey entries.\n")

# # === HANDLE TIMESTAMP COLUMN ===
# if "AccoDateStamp" in pred_df.columns:
#     pred_df.rename(columns={"AccoDateStamp": "AccoTime"}, inplace=True)

# # === EXTRACT INVOICE NUMBER FROM AccoDocNo (e.g., 224673/155 → 155) ===
# pred_df["InvNo"] = pred_df["AccoDocNo"].astype(str).apply(
#     lambda x: re.search(r"/(\d+)", x).group(1) if re.search(r"/(\d+)", x) else None
# )
# pred_df["InvNo"] = pd.to_numeric(pred_df["InvNo"], errors="coerce")

# print("🧾 Extracted InvNo from AccoDocNo — sample:")
# print(pred_df["InvNo"].dropna().head(), "\n")

# # === PREPARE SURVEY DATA ===
# survey_df.columns = survey_df.columns.str.strip()
# possible_inv_cols = [c for c in survey_df.columns if "inv" in c.lower()]
# inv_col = possible_inv_cols[0] if possible_inv_cols else "Inv No"
# print(f"✅ Using invoice column: '{inv_col}'")

# survey = survey_df.copy()
# survey.rename(columns={
#     inv_col: "InvNo",
#     "Sex": "survey_gender",
#     "Time": "survey_time"
# }, inplace=True, errors="ignore")

# # === COMBINE AGE BRACKETS (66–80 and 80+) ===
# if "80+" in survey.columns:
#     if "66–80" not in survey.columns:
#         survey["66–80"] = None
#     survey["66–80"] = survey[["66–80", "80+"]].apply(
#         lambda x: "X" if ("X" in str(x[0]).upper() or "X" in str(x[1]).upper()) else "",
#         axis=1
#     )
#     print("🔁 Merged '66–80' and '80+' into single bracket.")

# # === DETECT AGE GROUP FROM X MARKS ===
# age_columns = ["1–10", "11–20", "21–30", "31–40", "41–55", "56–65", "66–80"]
# survey["survey_age_group"] = survey[age_columns].apply(
#     lambda row: next((col for col in age_columns if str(row[col]).upper() == "X"), None),
#     axis=1
# )

# # === NORMALIZE FORMATTING ===
# merge_map = {
#     "1–10": "01–10", "11–20": "11–20", "21–30": "21–30",
#     "31–40": "31–40", "41–55": "41–55", "56–65": "56–65", "66–80": "66–80"
# }
# survey["survey_age_group"] = survey["survey_age_group"].map(merge_map)

# # === SELECT RELEVANT FIELDS ===
# survey = survey[["InvNo", "survey_gender", "survey_age_group", "survey_time"]]
# survey["InvNo"] = pd.to_numeric(survey["InvNo"], errors="coerce")

# print(f"✅ Survey cleaned — {len(survey)} valid records with InvNo.\n")

# # === MERGE (Many items to one survey) ===
# merged = pred_df.merge(survey, on="InvNo", how="left")

# # === RULE-BASED MATCHING FOR MULTIPLE RESPONDENTS ===
# def best_survey_for_item(row):
#     subset = survey[survey["InvNo"] == row["InvNo"]]
#     if len(subset) == 0:
#         return pd.Series({"survey_gender": None, "survey_age_group": None, "survey_time": None})
#     if len(subset) == 1:
#         return subset.iloc[0][["survey_gender", "survey_age_group", "survey_time"]]

#     # multiple respondents — use simple demographic match
#     subset = subset.copy()
#     subset["score"] = 0
#     subset.loc[subset["survey_gender"] == row["predicted_gender"], "score"] += 1
#     subset.loc[subset["survey_age_group"] == row["predicted_age"], "score"] += 1
#     best = subset.sort_values("score", ascending=False).iloc[0]
#     return best[["survey_gender", "survey_age_group", "survey_time"]]

# for col in ["survey_gender", "survey_age_group", "survey_time"]:
#     merged[col] = None

# merged[["survey_gender", "survey_age_group", "survey_time"]] = merged.apply(best_survey_for_item, axis=1)
# merged["survey_completed"] = merged["survey_gender"].notna()

# # === SAVE FINAL CSV ===
# merged.to_csv(output_csv, index=False, encoding="utf-8-sig")
# print(f"\n✅ Final merged file saved: {output_csv}")
# print(f"📊 Total rows: {len(merged)} | Completed surveys: {merged['survey_completed'].sum()}\n")
# display(merged.head(10))


🔍 Detected encoding for predictions CSV: utf-8
✅ Loaded 735 prediction records (items) and 64 survey entries.

🧾 Extracted InvNo from AccoDocNo — sample:
0    14.0
1    14.0
2    14.0
3    14.0
4    14.0
Name: InvNo, dtype: float64 

✅ Using invoice column: 'Inv No'
🔁 Merged '66–80' and '80+' into single bracket.
✅ Survey cleaned — 64 valid records with InvNo.


✅ Final merged file saved: ../results/20251107/results_faces_accounting_20251107_vit_final.csv
📊 Total rows: 829 | Completed surveys: 294



,image_name,cluster_id,AccoID,AccoDocNo,AccoDate,timestamp,AccoAmount,Quantity,Discount,UnitPrice,...,StockDesciption,predicted_gender,confidence,predicted_age,age_confidence,InvNo,survey_gender,survey_age_group,survey_time,survey_completed
0,cluster_000_AccoID_1170854_20251107_091623.png,0,1170854,224882/14,2025-11-07 09:17:00,2025-11-07 09:16:23,-460.0,1.0,0.0,29.0,...,wors breakfast,Female,0.999,21–30,0.742,14.0,F,41–55,08:57:00,True
1,cluster_000_AccoID_1170854_20251107_091623.png,0,1170854,224882/14,2025-11-07 09:17:00,2025-11-07 09:16:23,-460.0,1.0,0.0,39.0,...,plaas koffee,Female,0.999,21–30,0.742,14.0,F,41–55,08:57:00,True
2,cluster_000_AccoID_1170854_20251107_091623.png,0,1170854,224882/14,2025-11-07 09:17:00,2025-11-07 09:16:23,-460.0,1.0,0.0,20.0,...,americano black,Female,0.999,21–30,0.742,14.0,F,41–55,08:57:00,True
3,cluster_000_AccoID_1170854_20251107_091623.png,0,1170854,224882/14,2025-11-07 09:17:00,2025-11-07 09:16:23,-460.0,1.0,0.0,29.0,...,cappuccino,Female,0.999,21–30,0.742,14.0,F,41–55,08:57:00,True
4,cluster_000_AccoID_1170854_20251107_091623.png,0,1170854,224882/14,2025-11-07 09:17:00,2025-11-07 09:16:23,-460.0,1.0,0.0,59.0,...,lite,Female,0.999,21–30,0.742,14.0,F,41–55,08:57:00,True
5,cluster_000_AccoID_1170854_20251107_091623.png,0,1170854,224882/14,2025-11-07 09:17:00,2025-11-07 09:16:23,-460.0,2.0,0.0,53.0,...,pbb peanut butter,Female,0.999,21–30,0.742,14.0,F,41–55,08:57:00,True
6,cluster_000_AccoID_1170854_20251107_091623.png,0,1170854,224882/14,2025-11-07 09:17:00,2025-11-07 09:16:23,-460.0,2.0,0.0,89.0,...,hadeda,Female,0.999,21–30,0.742,14.0,F,41–55,08:57:00,True
7,cluster_001_AccoID_1171262_20251107_112821.png,1,1171262,224949/77,2025-11-07 11:28:00,2025-11-07 11:28:21,-112.0,4.0,0.2,35.0,...,beef patty,Female,0.970,31–40,0.448,77.0,None,None,None,False
8,cluster_002_AccoID_1171659_20251107_135038.png,2,1171659,225013/151,2025-11-07 13:51:00,2025-11-07 13:50:38,-85.0,1.0,0.0,36.0,...,cappuccino,Female,0.948,21–30,0.543,151.0,None,None,None,False
9,cluster_002_AccoID_1171659_20251107_135038.png,2,1171659,225013/151,2025-11-07 13:51:00,2025-11-07 13:50:38,-85.0,1.0,0.0,49.0,...,just pops,Female,0.948,21–30,0.543,151.0,None,None,None,False


In [43]:
import pandas as pd
import re
import chardet

# === PATHS ===
predictions_csv = "../results/20251107/results_faces_accounting_20251107_vit.csv"
survey_xlsx = "../survey/Survey_Cafe_7-11-2025.xlsx"
output_csv = "../results/20251107/results_faces_accounting_20251107_vit_final.csv"

# === LOAD WITH SAFE ENCODING DETECTION ===
with open(predictions_csv, "rb") as f:
    enc = chardet.detect(f.read())["encoding"]
print(f"🔍 Detected encoding for predictions CSV: {enc}")

pred_df = pd.read_csv(predictions_csv, encoding=enc, on_bad_lines="skip", low_memory=False)
survey_df = pd.read_excel(survey_xlsx)

print(f"✅ Loaded {len(pred_df)} prediction records and {len(survey_df)} survey entries.\n")

# === EXTRACT InvNo FROM AccoDocNo ===
if "AccoDocNo" in pred_df.columns:
    pred_df["InvNo"] = pred_df["AccoDocNo"].astype(str).str.extract(r"/(\d+)", expand=False)
pred_df["InvNo"] = pd.to_numeric(pred_df["InvNo"], errors="coerce")

# === NORMALIZE SURVEY COLUMN NAMES ===
def normalize_col(c: str) -> str:
    c = str(c).strip()
    c = re.sub(r"\s+", " ", c.replace("\xa0", " "))
    c = c.replace("–", "-").replace("—", "-")
    return c

survey_df.columns = [normalize_col(c) for c in survey_df.columns]

# === FIND AND RENAME INVOICE COLUMN ===
inv_candidates = [c for c in survey_df.columns if "inv" in c.lower()]
if not inv_candidates:
    raise KeyError("❌ No invoice column found in survey sheet.")
inv_col = inv_candidates[0]
if inv_col != "InvNo":
    survey_df.rename(columns={inv_col: "InvNo"}, inplace=True)

# === RENAME STANDARD COLUMNS ===
rename_map = {}
if "Sex" in survey_df.columns:
    rename_map["Sex"] = "survey_gender"
if "Time" in survey_df.columns:
    rename_map["Time"] = "survey_time"
survey_df.rename(columns=rename_map, inplace=True)

# === NORMALIZE AGE COLUMNS ===
canonical_age_headers = ["1-10", "11-20", "21-30", "31-40", "41-55", "56-65", "66-80", "80+"]
age_cols = [c for c in survey_df.columns if c in canonical_age_headers]
if not age_cols:
    # fallback pattern for dash variants
    age_cols = [c for c in survey_df.columns if re.match(r"^\d+\s*[-–—]\s*\d+$", c) or c.strip().endswith("+")]
if not age_cols:
    raise KeyError("❌ Could not find any age columns in survey sheet.")

# === MERGE 66–80 and 80+ ===
if "80+" in survey_df.columns:
    if "66-80" not in survey_df.columns:
        survey_df["66-80"] = ""
    survey_df["66-80"] = survey_df[["66-80", "80+"]].apply(
        lambda x: "X" if ("X" in str(x[0]).upper() or "X" in str(x[1]).upper()) else "",
        axis=1
    )

# === DETECT survey_age_group FROM X ===
def pick_age(row):
    for c in age_cols:
        if str(row[c]).strip().upper() == "X":
            return c
    return None

survey_df["survey_age_group"] = survey_df.apply(pick_age, axis=1)

# === KEEP ONLY RELEVANT COLUMNS ===
cols = ["InvNo", "survey_gender", "survey_age_group"]
if "survey_time" in survey_df.columns:
    cols.append("survey_time")

survey = survey_df[cols].copy()
survey["InvNo"] = pd.to_numeric(survey["InvNo"], errors="coerce")

# === NORMALIZE GENDER TEXT ===
survey["survey_gender"] = (
    survey["survey_gender"]
    .astype(str).str.strip().str.upper()
    .replace({"M": "Male", "MALE": "Male", "F": "Female", "FEMALE": "Female"})
)

# === MERGE PREDICTIONS WITH SURVEY ===
merged = pred_df.merge(survey, on="InvNo", how="left")

# === HANDLE MULTIPLE SURVEYS PER INVOICE ===
def best_survey_for_item(row):
    subset = survey[survey["InvNo"] == row["InvNo"]]
    if len(subset) == 0:
        return pd.Series({"survey_gender": None, "survey_age_group": None, "survey_time": None})
    if len(subset) == 1:
        return subset.iloc[0][["survey_gender", "survey_age_group", "survey_time"]]
    subset = subset.copy()
    subset["score"] = 0
    if "predicted_gender" in row:
        subset.loc[subset["survey_gender"] == row["predicted_gender"], "score"] += 1
    if "predicted_age" in row:
        subset.loc[subset["survey_age_group"] == row["predicted_age"], "score"] += 1
    best = subset.sort_values("score", ascending=False).iloc[0]
    return best[["survey_gender", "survey_age_group", "survey_time"]]

# Apply selection
replacement = merged.apply(best_survey_for_item, axis=1)
for c in ["survey_gender", "survey_age_group", "survey_time"]:
    merged[c] = replacement[c]

# === ADD SURVEY COMPLETION FLAG ===
merged["survey_completed"] = merged["survey_gender"].notna()

# === SAVE FINAL FILE ===
merged.to_csv(output_csv, index=False, encoding="utf-8-sig")
print(f"✅ Saved merged file: {output_csv}")
print(f"📊 Total rows: {len(merged)} | Completed surveys: {merged['survey_completed'].sum()}")

# === SUMMARY CHECK ===
print("\n📊 Survey Gender Distribution:")
print(merged["survey_gender"].value_counts(dropna=False))


🔍 Detected encoding for predictions CSV: utf-8
✅ Loaded 735 prediction records and 61 survey entries.

✅ Saved merged file: ../results/20251107/results_faces_accounting_20251107_vit_final.csv
📊 Total rows: 894 | Completed surveys: 384

📊 Survey Gender Distribution:
survey_gender
None      510
Female    300
Male       84
Name: count, dtype: int64


In [1]:
import pandas as pd
from pathlib import Path

# Paths to your four original processed CSVs
csv1 = "../results/20251105/results_faces_accounting_20251105_final.csv"
csv2 = "../results/20251107/results_faces_accounting_20251107_final.csv"

csv3 = "../results/20251105/results_faces_accounting_20251105_vit_final.csv"
csv4 = "../results/20251107/results_faces_accounting_20251107_vit_final.csv"

# -------------------------------------------------------------------
# Load safely (automatic UTF-8 fallback)
# -------------------------------------------------------------------
def safe_read(path):
    try:
        return pd.read_csv(path, encoding="utf-8")
    except UnicodeDecodeError:
        return pd.read_csv(path, encoding="cp1252")

# Read all four
df_05_final = safe_read(csv1)
df_07_final = safe_read(csv2)

df_05_vit = safe_read(csv3)
df_07_vit = safe_read(csv4)

# -------------------------------------------------------------------
# Combine them
# -------------------------------------------------------------------
combined_final = pd.concat([df_05_final, df_07_final], ignore_index=True)
combined_vit_final = pd.concat([df_05_vit, df_07_vit], ignore_index=True)

# -------------------------------------------------------------------
# Export clean, UTF-8 encoded combined datasets
# -------------------------------------------------------------------
output_dir = Path("../results/combined_dates")
output_dir.mkdir(parents=True, exist_ok=True)

combined_final_path = output_dir / "combined_dates_final.csv"
combined_vit_final_path = output_dir / "combined_dates_vit_final.csv"

combined_final.to_csv(combined_final_path, index=False, encoding="utf-8")
combined_vit_final.to_csv(combined_vit_final_path, index=False, encoding="utf-8")

print("✔ Combined CSVs created:")
print(" -", combined_final_path)
print(" -", combined_vit_final_path)


✔ Combined CSVs created:
 - ..\results\combined_dates\combined_dates_final.csv
 - ..\results\combined_dates\combined_dates_vit_final.csv
